In [164]:
import pandas as pd
from sklearn.ensemble import AdaBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeRegressor
from sklearn import metrics
import numpy as np

## DecisionTreeRegressor

Treinando o modelo


In [179]:
df_train = pd.read_csv('data/train.csv')
X_train = df_train[['GrLivArea','OverallQual']]
#X_train = pd.get_dummies(X_train, columns = ['OverallQual'],prefix='OverallQual')
y_train = df_train['SalePrice']
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.33, random_state=42)

Treinando o modelo

In [166]:
reg = DecisionTreeRegressor()
reg.fit(X_train,y_train)
print(f'Modelo AdaBoost Score= {100*reg.score(X_train, y_train)}')

Modelo AdaBoost Score= 98.14086066841958


Realizando predicted

In [167]:
y_predict = reg.predict(X_test)

print('MAE:', metrics.mean_absolute_error(y_test,y_predict))
print('MSE:', metrics.mean_squared_error(y_test, y_predict))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, y_predict)))


MAE: 34566.616182572616
MSE: 3450207115.7294836
RMSE: 58738.463682066824


Utilizando GridSearchCV para Parâmetros

In [183]:
parameters={"splitter":["best","random"],
            "max_depth" : [1,3,5,7,9,11,12],
           "min_samples_leaf":[1,2,3,4,5,6,7,8,9,10],
           "min_weight_fraction_leaf":[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9],
           "max_features":["auto","log2","sqrt",None] ,
           "max_leaf_nodes":[None,10,20,30,40,50,60,70,80,90] }

from sklearn.model_selection import GridSearchCV
tuning_model=GridSearchCV(reg,param_grid=parameters,scoring='neg_mean_squared_error',cv=3,verbose=3)
tuning_model.fit(X_train,y_train)

In [169]:
tuning_model.best_params_


{'max_depth': 11,
 'max_features': 'log2',
 'max_leaf_nodes': None,
 'min_samples_leaf': 8,
 'min_weight_fraction_leaf': 0.1,
 'splitter': 'best'}

In [180]:
reg = DecisionTreeRegressor(max_depth= 11,
             max_features= 'log2',
             max_leaf_nodes= None,
             min_samples_leaf= 8,
             min_weight_fraction_leaf= 0.1,
             splitter= 'best')
reg.fit(X_train,y_train)
print(f'Modelo AdaBoost Score= {100*reg.score(X_train, y_train)}')

Modelo AdaBoost Score= 62.34112910831524


In [181]:
y_predict = reg.predict(X_test)
print('MAE:', metrics.mean_absolute_error(y_test,y_predict))
print('MSE:', metrics.mean_squared_error(y_test, y_predict))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, y_predict)))




MAE: 33304.64246002283
MSE: 2880387559.7162366
RMSE: 53669.24221298673


## Enviando submissão

In [182]:
df_test = pd.read_csv('data/test.csv')
X_test = df_test[['GrLivArea','OverallQual']]
#X_test = pd.get_dummies(X_test, columns = ['OverallQual'],prefix='OverallQual')
y_predict = reg.predict(X_test)

dict = {
    'Id' : df_test.Id,
    'SalePrice' : y_predict
}
df = pd.DataFrame(dict)
df.to_csv('predicted.csv',index=False)